In [1]:
%load_ext autoreload
%autoreload 2

from time import sleep
import pandas as pd
from tqdm import tqdm
from scraper import get_program_links
from scraper import extract_program_details
import csv


# Base URL to handle relative links
BASE_URL = "https://www.foerderdatenbank.de/"

In [3]:
start_url = "https://www.foerderdatenbank.de/SiteGlobals/FDB/Forms/Suche/Foederprogrammsuche_Formular.html?input_=23adddb0-dcf7-4e32-96f5-93aec5db2716&submit=Suchen&resourceId=0065e6ec-5c0a-4678-b503-b7e7ec435dfd&filterCategories=FundingProgram&pageLocale=de"

# check if there is a cache of the collected links (data/program_links.txt) and load from there.
try:
    with open("data/program_links.txt", "r") as f:
        program_links = f.read().splitlines()
except FileNotFoundError:
    program_links = get_program_links(url=start_url, BASE_URL=BASE_URL)

# Print the collected links
for link in program_links:
    print(link)

https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/Sachsen/buergschaft-sachsen-beteiligung.html
https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/NRW/garantien-von-beteiligungen-an-kmu-der-gewerbliche.html
https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/Sachsen/foerderrichtlinie-ausgleichszulage-sachsen.html
https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/Sachsen/rettung-umstrukturier-kleine-mittlere-unternehmen.html
https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/Sachsen/investitionen-teilhabe.html
https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/Sachsen/foerderung-der-kommunale-praevention-rl-komm-praev.html
https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/Sachsen/foerderung-sicherung-weiterentwickl-qualitaet-kita.html
https://www.foerderdatenbank.de/FDB/Content/DE/Foerderprogramm/Land/Thueringen/guw-thueringen.html
https://www.foerderdatenbank.de/FDB/C

In [4]:
# write the collected links to a file, so we don't have to scrape them again.
with open("data/program_links.txt", "w") as f:
    f.write("\n".join(program_links))

In [15]:
try:
    with open("data/funding_programs.csv", "r") as f:
        reader = csv.DictReader(f)
        programs = list(reader)
        program_details = {row["url"]: row for row in programs}
        for p in program_details:
            program_details[p].pop('url', None)
except FileNotFoundError:
    program_details = {}

for link in tqdm(program_links):
    if link in program_details:
        continue
    program_details[link] = extract_program_details(url=link)
    sleep(1)  # Be nice to the server

100%|██████████| 2353/2353 [47:42<00:00,  1.22s/it]


In [16]:
funding_programs = pd.DataFrame.from_dict(program_details, orient='index')

In [17]:
funding_programs.to_csv("data/funding_programs.csv", index_label="url")